# Block-wise viewing of an Image
Stough, DIP

Through spatial filtering we have been looking at ways of understanding the local neighborhood of a pixel. We saw that a low-pass spatial filter such as a Gaussian can give us some kind of weighted average of the local neighborhood, while the high-pass (e.g., Laplacian) tells us the degree to which a pixel is on an edge, or the edginess of the neighborhood. 

## Block Transforms
To simplify our studies, we're going to start splitting up the image into non-overlapping square blocks. That way, instead of having a different local neighborhood for each pixel, we'll now have every pixel within a block sharing the local neighborhood. Let's see what we mean.

- [View as blocks](https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.view_as_blocks)
- [Montage](https://scikit-image.org/docs/0.7.0/api/skimage.util.montage.html#)
- [Online example of usage](https://scikit-image.org/docs/dev/auto_examples/numpy_operations/plot_view_as_blocks.html)

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from skimage.util import view_as_blocks
from skimage.util import montage

In [2]:
def normImage(I):
    I = I.copy() - I.min()
    I = I/I.max()
    return I

def arrInfo(I):
    return I.shape, I.min(), I.max(), I.dtype

In [3]:
# Load an image
I = plt.imread('candy.png')
arrInfo(I)

((800, 1200, 3), 0.0, 1.0, dtype('float32'))

&nbsp;
## Split the image into 8x8x3 blocks

In [4]:
block_shape = (8, 8, 3)
view = view_as_blocks(I, block_shape=block_shape)

In [5]:
view.shape

(100, 150, 1, 8, 8, 3)

In [6]:
# We'll reshape the view so we can process the blocks in a single dimension.
view = np.squeeze(view)

blockView = view.reshape([view.shape[0]*view.shape[1]] + list(view.shape[2:]))
print(blockView.shape)

(15000, 8, 8, 3)


&nbsp;

## Let's replace each block with its mean color
Study the below loop. I know I said we'd avoid looping over pixels, but I kind of lied. Here, we're looping over 8x8 blocks, which is okay :-P

Think about how we can compute almost anything about a block inside this loop, 

In [7]:
newImageBlocks = np.zeros(blockView.shape)

for i, block in enumerate(blockView):
    bT = np.mean(block, axis=(0,1)) # Some transform of the block
    # bT is a (3,) array of the average color of the block
    # This line sets each of the 8x8 pixels to be the (1,1,3) version of the bT
    newImageBlocks[i][:] = np.reshape(bT, (1,1,3)) 

In [8]:
# Use montage to put the blocks back together.
I_mean = montage(newImageBlocks, grid_shape=[view.shape[0], view.shape[1]], multichannel=True)

In [9]:
arrInfo(I_mean)

((800, 1200, 3), 0.05061276629567146, 0.9792894721031189, dtype('float64'))

&nbsp;

## Visualize the image and its mean reconstruction

In [10]:
f, ax = plt.subplots(1,2, figsize=(7,3), sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title('Original Image')
ax[1].imshow(I_mean)
ax[1].set_title('Mean Reconstruction')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

&nbsp;

## Characterizing the information in a block 
In the above we've replaced the 64 colors within a block with just one color. But then, if you don't zoom in too much, notice that we didn't lose a lot of what matters about the image. In fact, if you could keep just one piece of information about a block, you could do a lot worse than the mean. 

In fact what we just did is 64:1 lossy compression of the image. That's great, but with quite a bit of loss. Maybe we could keep more information in places where it matters, and less in places where it doesn't.

In [11]:
(1,1)[0]

1

In [12]:
'''
myFucntion takes an image I and a block_shape which is a tuple (8,8)
and will return a lossy compression of the image using the blocks mean
'''
def myFunction(I, block_shape):
    block_shape = (block_shape[0], block_shape[1], 3)
    view = view_as_blocks(I, block_shape=block_shape)
    # We'll reshape the view so we can process the blocks in a single dimension.
    view = np.squeeze(view)

    blockView = view.reshape([view.shape[0]*view.shape[1]] + list(view.shape[2:]))
    print(blockView.shape)
    
    newImageBlocks = np.zeros(blockView.shape)

    for i, block in enumerate(blockView):
        bT = np.mean(block, axis=(0,1)) # Some transform of the block
        # bT is a (3,) array of the average color of the block
        # This line sets each of the 8x8 pixels to be the (1,1,3) version of the bT
        newImageBlocks[i][:] = np.reshape(bT, (1,1,3)) 
    I_mean = montage(newImageBlocks, grid_shape=[view.shape[0], view.shape[1]], multichannel=True)
    return I_mean

In [13]:
M = myFunction(M, (8,8))
#arrInfo(M)
plt.figure()
plt.imshow(M)

NameError: name 'M' is not defined

In [14]:
def block_transform(I, block_shape,transform='mean'):
    block_shape = (block_shape[0], block_shape[1], 3)
    view = view_as_blocks(I, block_shape=block_shape)
    # We'll reshape the view so we can process the blocks in a single dimension.
    view = np.squeeze(view)

    blockView = view.reshape([view.shape[0]*view.shape[1]] + list(view.shape[2:]))
    print(blockView.shape)
    
    newImageBlocks = np.zeros(blockView.shape)

    for i, block in enumerate(blockView):
        if(transform=='mean'):
            bT = np.mean(block, axis=(0,1)) # Mean transform of the block
        elif(transform == 'max'):
            bT = np.max(block, axis=(0,1)) # Some transform of the block
        elif(transform =="median"):
            bT = np.median(block, axis=(0,1))
        elif(transform == "min"):
             bT= np.min(block,axis=(0,1))
        else:
            print("Unknown transform returning -1")
            return -1
        # bT is a (3,) array of the average color of the block
        # This line sets each of the 8x8 pixels to be the (1,1,3) version of the bT
        newImageBlocks[i][:] = np.reshape(bT, (1,1,3)) 
    I_max = montage(newImageBlocks, grid_shape=[view.shape[0], view.shape[1]], multichannel=True)
    return I_max

In [15]:

f, ax = plt.subplots(1,4, sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title("Original")
ax[1].imshow(block_transform(I,(8,8),"median"))
ax[1].set_title("Median")
ax[2].imshow(block_transform(I,(8,8),"max"))
ax[2].set_title("Max")
ax[3].imshow(block_transform(I,(8,8),"min"))
ax[3].set_title("Min")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(15000, 8, 8, 3)
(15000, 8, 8, 3)
(15000, 8, 8, 3)


Text(0.5, 1.0, 'Min')

In [16]:
f, ax = plt.subplots(1,3, sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title("Original")
ax[1].imshow(block_transform(I,(8,8),"mean"))
ax[1].set_title("8x8 mean")
ax[2].imshow(block_transform(I,(4,4),"mean"))
ax[2].set_title("4x4 mean")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(15000, 8, 8, 3)
(60000, 4, 4, 3)


Text(0.5, 1.0, '4x4 mean')